In [1]:
import numpy as np
import pandas as pd 

In [2]:
use_cols = ["BUYER_STATE","BUYER_COUNTY", "CALC_BASE_WT_IN_GM", "MME_Conversion_Factor","TRANSACTION_DATE", "DRUG_NAME"]
import os
os.chdir('/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2')
os.getcwd()


'/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2'

In [3]:
NM_pre_policy = pd.read_csv("./20_intermediate_files/NMpop2000.csv", header = [3])
NM_post_policy = pd.read_csv("./20_intermediate_files/NMpop2010.csv", header = [3])

In [4]:
def clean_pop(df1,df2):
    df1 = df1.drop(["Unnamed: 1", "2000", "2001", "2002", "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", "2016", "2017", "2018", "2019"], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    return df_concat

In [5]:
NM_total_pop = clean_pop(NM_pre_policy, NM_post_policy)

In [6]:

NM_shipment = pd.read_csv('./20_intermediate_files/arcos-nm-statewide-itemized.csv.gz',  compression='gzip', usecols = use_cols,
                chunksize = 1000000,
                error_bad_lines=False)

In [7]:
df1 = pd.DataFrame()

for i in NM_shipment:
    a = pd.DataFrame(i)
    df1 = df1.append(a)

In [8]:
df1['TRANSACTION_YEAR'] = df1['TRANSACTION_DATE'].astype(str).str[-4:] #get the year from this column
        
df1['CALC_BASE_WT_IN_MG'] = df1['CALC_BASE_WT_IN_GM'] * 1000 # convert to milligrams

df1['MORPHINE_EQUIV_IN_MG'] = df1['CALC_BASE_WT_IN_MG'] * df1['MME_Conversion_Factor']



final_shipments_NM = df1[["TRANSACTION_YEAR", "BUYER_STATE", "BUYER_COUNTY", "MORPHINE_EQUIV_IN_MG"]]

final_shipments_NM = final_shipments_NM.rename(columns={"BUYER_COUNTY": "County", "TRANSACTION_YEAR":"Year"})

final_shipments_NM[["Year"]] = final_shipments_NM["Year"].astype(int)

In [9]:
NM_total_pop["County"] = NM_total_pop["County"].str.upper() #make entire column upper case 
NM_total_pop['County'] = NM_total_pop['County'].astype(str).str[:-7] # remove "county" from county name 

In [10]:
final_shipments_NM['County'] = final_shipments_NM['County'].astype(str)
NM_total_pop['County'] = NM_total_pop['County'].astype(str)

NM_total_pop['Year'] = NM_total_pop['Year'].astype(int)
final_shipments_NM['Year'] = final_shipments_NM['Year'].astype(int)

In [11]:
mergeddf = final_shipments_NM.merge(NM_total_pop, on= ["Year", "County"],  how='left')
mergeddf

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2014,NM,DE BACA,4540.50,"1,861"
1,2014,NM,DE BACA,2270.25,"1,861"
2,2014,NM,DE BACA,3027.00,"1,861"
3,2012,NM,VALENCIA,1513.50,"76,789"
4,2014,NM,VALENCIA,3027.00,"75,892"
...,...,...,...,...,...
1540142,2009,NM,DONA ANA,24205.50,"205,401"
1540143,2006,NM,BERNALILLO,1513.50,"628,632"
1540144,2009,NM,OTERO,3027.00,"62,462"
1540145,2007,NM,DONA ANA,302.70,"197,853"


In [12]:
mergeddf.to_csv(r"/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/NM_shipment_and_pop.csv")